## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,60.48,39,91,3.83,overcast clouds
1,1,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,60.28,72,100,5.37,overcast clouds
2,2,Podgornoye,RU,51.7350,39.1496,43.02,81,75,11.18,broken clouds
3,3,Lerwick,GB,60.1545,-1.1494,50.00,84,99,22.75,overcast clouds
4,4,La Asuncion,VE,11.0333,-63.8628,78.69,87,60,12.91,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,La Asuncion,VE,11.0333,-63.8628,78.69,87,60,12.91,broken clouds
9,9,Rikitea,PF,-23.1203,-134.9692,75.52,83,98,13.85,overcast clouds
11,11,Coahuayana,MX,18.7333,-103.6833,77.14,90,100,2.15,overcast clouds
13,13,Sinnamary,GF,5.3833,-52.9500,77.86,88,100,7.54,overcast clouds
16,16,Kavaratti,IN,10.5669,72.6420,81.73,72,95,9.82,overcast clouds
18,18,Hilo,US,19.7297,-155.0900,80.33,66,40,3.44,scattered clouds
19,19,Bowen,AU,-20.0167,148.2333,81.97,61,75,11.21,broken clouds
23,23,Rio Hato,PA,8.3833,-80.1667,80.78,86,49,5.61,scattered clouds
28,28,Alofi,NU,-19.0595,-169.9187,82.29,69,58,14.97,broken clouds
29,29,Victoria,HK,22.2855,114.1577,86.05,78,5,3.00,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                178
City                   178
Country                178
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna() #In this case, no dropped rows

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,La Asuncion,VE,78.69,broken clouds,11.0333,-63.8628,
9,Rikitea,PF,75.52,overcast clouds,-23.1203,-134.9692,
11,Coahuayana,MX,77.14,overcast clouds,18.7333,-103.6833,
13,Sinnamary,GF,77.86,overcast clouds,5.3833,-52.9500,
16,Kavaratti,IN,81.73,overcast clouds,10.5669,72.6420,
18,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,
19,Bowen,AU,81.97,broken clouds,-20.0167,148.2333,
23,Rio Hato,PA,80.78,scattered clouds,8.3833,-80.1667,
28,Alofi,NU,82.29,broken clouds,-19.0595,-169.9187,
29,Victoria,HK,86.05,clear sky,22.2855,114.1577,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,La Asuncion,VE,78.69,broken clouds,11.0333,-63.8628,Colinas de Matasiete
9,Rikitea,PF,75.52,overcast clouds,-23.1203,-134.9692,People ThankYou
11,Coahuayana,MX,77.14,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
13,Sinnamary,GF,77.86,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
16,Kavaratti,IN,81.73,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [12]:
len(clean_hotel_df)

163

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))